
**本notebookは以下のリンクで解説されているチュートリアルをもとに、日本語にしたものです。**
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/pix2pix.ipynb

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# pix2pix: Image-to-image translation with a conditional GAN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/pix2pix"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/pix2pix.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/pix2pix.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/generative/pix2pix.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

　 このチュートリアルでは、Phillip Isola等によって２０１７年に発表された[Image-to-image translation with conditional adversarial networks](https://arxiv.org/abs/1611.07004)に記載されている、入力画像と出力画像に相当する２つの画像の対応関係を学習し、変換アルゴリズムを獲得できるpix2pixという条件付き敵対生成ネットワーク（Conditional GAN）モデルを構築し、学習させる方法を示します。pix2pixは以下の図から分かる通り、用途を限定せず、セグメンテーション画像から写真を合成したり、白黒画像からカラー化された写真を生成したり、Googleマップの写真を航空写真にしたり、スケッチを写真にしたりと、幅広い用途に対応しています。

<br><br>![teaser_v3](https://phillipi.github.io/pix2pix/images/teaser_v3.png)<br><br>

  このColab notebookでは、プラハにある[チェコ工科大学](https://www.cvut.cz/)の[Center for Machine Perception](http://cmp.felk.cvut.cz/)が提供している[CMP Facade Database](http://cmp.felk.cvut.cz/~tylecr1/facade/) を使って、建物の外観（facades：ファサード）画像を生成します。データの事前準備を省略するものとして、pix2pixの作者が作成した[前処理済みデータセット]((https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/)を使用します。


  pix2pixのcGANでは、入力画像を条件として、それに対応する出力画像を生成します。cGANは、[Conditional Generative Adversarial Nets](https://arxiv.org/abs/1411.1784)（Mirza and Osindero, 2014）で初めて提案されました


  ネットワークアーキテクチャ（内部構造）には以下のものが含まれています。

- 生成器として、U-Netベースのモデル。
- 識別器として、論文内で提案されているPatchGAN（畳み込み層をもつ）。

※　なお，V100 GPUを1台使用した場合，1回のエポックに約15秒ほどかかります。


以下の画像は，ファサードデータセット（80k step）で200epochの学習を行った後， pix2pix　（cGAN）　が生成した出力結果です．
<br><br>
![sample output_1](https://www.tensorflow.org/images/gan/pix2pix_1.png)
![sample output_2](https://www.tensorflow.org/images/gan/pix2pix_2.png)


## 必要なライブラリ類をインストール（Tensorflow等）

In [ ]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

from matplotlib import pyplot as plt
from IPython import display

## データセット準備

まず、CMP Facade Database data (30MB)をダウンロードします。その他のデータセットも同じデータフォーマットで[こちら](http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/)から入手できます。  以下のセル内では、ドロップダウンメニューから他のデータセットを選択し、ダウンロードできるようにしてあります。他のデータセットの中には、かなり大きいものもありますのでご注意ください。例えば、`edges2handbags` は 8GBもあります。

In [ ]:
dataset_name = "facades" #@param ["cityscapes", "edges2handbags", "edges2shoes", "facades", "maps", "night2day"]


In [ ]:
_URL = f'http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{dataset_name}.tar.gz'

path_to_zip = tf.keras.utils.get_file(
    fname=f"{dataset_name}.tar.gz",
    origin=_URL,
    extract=True)

path_to_zip  = pathlib.Path(path_to_zip)

PATH = path_to_zip.parent/dataset_name

In [ ]:
list(PATH.parent.iterdir())

各データセット画像のサイズは `256 x 256`の画像を二つ横に並べて、`256 x 512` のサイズで用意されております。

In [ ]:
sample_image = tf.io.read_file(str(PATH / 'train/1.jpg'))
sample_image = tf.io.decode_jpeg(sample_image)
print(sample_image.shape)

In [ ]:
plt.figure()
plt.imshow(sample_image)

もし、あなたが実際の建物の外観画像と正解のセグメンテーション画像を`256 x 256`サイズの画像に分割したいのであれば、データセットの画像ファイルを読み込んで、2つの画像にテンソルで分割する以下のような関数を定義する必要があります。

In [ ]:
def load(image_file):
  # Read and decode an image file to a uint8 tensor
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  # Split each image tensor into two tensors:
  # - one with a real building facade image
  # - one with an architecture label image 
  w = tf.shape(image)[1]
  w = w // 2
  input_image = image[:, w:, :]
  real_image = image[:, :w, :]

  # Convert both images to float32 tensors
  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

入力画像（建物のセグメンテーション画像）と実画像（建物の外観写真）の2つの画像に分けたものを描画します。

In [ ]:
inp, re = load(str(PATH / 'train/100.jpg'))
# Casting to int for matplotlib to display the images
plt.figure()
plt.imshow(inp / 255.0)
plt.figure()
plt.imshow(re / 255.0)

[元論文](https://arxiv.org/abs/1611.07004)で言及されているように、データ拡張として学習データセットにランダムにjittering とmirroringを適用する必要があります。

データ拡張をするために次のような関数を定義します。

1. `256 x 256`の各画像を，より大きな高さと幅（`286 x 286`）にリサイズする．
2. `256 x 256`にランダムにクロップする．
3. ランダムに画像を左右に反転させます。（ random jittering and mirroring ）
4. 画像を `[-1, 1]` の範囲に正規化する。

In [ ]:
# The facade training set consist of 400 images
BUFFER_SIZE = 400
# The batch size of 1 produced better results for the U-Net in the original pix2pix experiment
BATCH_SIZE = 1
# Each image is 256x256 in size
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [ ]:
# Normalizing the images to [-1, 1]
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
  # Resizing to 286x286
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # Random cropping back to 256x256
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # Random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

データ拡張された出力の一部をプロットして、見ることができます。

In [ ]:
plt.figure(figsize=(6, 6))
for i in range(4):
  rj_inp, rj_re = random_jitter(inp, re)
  plt.subplot(2, 2, i + 1)
  plt.imshow(rj_inp / 255.0)
  plt.axis('off')
plt.show()

読み込みと前処理のデータ拡張が動作することを確認した後、学習セットとテストセットの読み込みとデータ拡張の前処理を行ういくつかのヘルパー関数を定義してみましょう。

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

## `tf.data` を使って、入力データを作成

In [ ]:
train_dataset = tf.data.Dataset.list_files(str(PATH / 'train/*.jpg'))
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
try:
  test_dataset = tf.data.Dataset.list_files(str(PATH / 'test/*.jpg'))
except tf.errors.InvalidArgumentError:
  test_dataset = tf.data.Dataset.list_files(str(PATH / 'val/*.jpg'))
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

## 生成器（ジェネレーター）

pix2pix(cGAN)の生成器は、[U-Net](https://arxiv.org/abs/1505.04597)を改造したものです。U-Netは、Encoder (Downsampler)とDecoder (Upsampler)で構成されています。
(詳細は、[セグメンテーションのチュートリアル](https://www.tensorflow.org/tutorials/images/segmentation)や[U-Net project website](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/)で確認できます)。

- Encoder (Downsampler)　内のブロック構造：`畳み込み層 -> バッチ正規化 -> Leaky ReLU`
- Decoder (Upsampler)内のブロック構造：`転置畳み込み層 -> バッチ正規化 -> ドロップアウト (最初の３ブロックはここまで) -> ReLU`
- EncoderとDecoderの間には、U-Netのようなスキップ結合があります。

Downsampler (Encoder) を定義します

In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

Upsampler (Decoder) を定義します

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

先程定義したDownsamplerとUpsamplerを用いて、生成器（ジェネレーター）を作成します

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256, 256, 3])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
    downsample(128, 4),  # (batch_size, 64, 64, 128)
    downsample(256, 4),  # (batch_size, 32, 32, 256)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
    downsample(512, 4),  # (batch_size, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(512, 4),  # (batch_size, 16, 16, 1024)
    upsample(256, 4),  # (batch_size, 32, 32, 512)
    upsample(128, 4),  # (batch_size, 64, 64, 256)
    upsample(64, 4),  # (batch_size, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

生成器（ジェネレーター）の可視化

In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

生成器（ジェネレーター）をテストする

In [ ]:
gen_output = generator(inp[tf.newaxis, ...], training=False)
plt.imshow(gen_output[0, ...])

### 生成器（ジェネレーター）の損失関数

GANは学習データに適応しているロス(損失値)を算出しますが、cGANは[pix2pixの論文](https://arxiv.org/abs/1611.07004)にあるように、ネットワークモデルの出力とターゲット画像が異なる可能性のあるモデル構造にペナルティを与えるような仕組みのロス（損失値）を用いて、学習します。以下に各損失値の詳細を述べます。

- 生成器のロス(損失値)は、生成された画像と**1の配列(生成画像を全て本物であると識別して欲しいために最初は本物の指標である１とおく)**のsigmoid cross-entropyロス（損失値）です。

- また、pix2pixの論文では、L1損失と表現されていますが、これは生成画像とターゲット画像間のMAE（平均絶対誤差）のことです。
　※ちなみに、L2損失はMSE（平均二乗誤差）のことです。
- L1損失(MAE)により、生成画像がターゲット画像が構造的に類似する事が期待されます。
- 生成器（ジェネレーター）ののトータルロス（損失値）を計算する式は、`gan_loss + LAMBDA * l1_loss` であり、論文の著者が`LAMBDA = 100` と指定している。

In [ ]:
LAMBDA = 100

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # Mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

以下に図として生成モデルの訓練時の流れを示す。

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)


## 識別器（ディスクリミネーター)

pix2pix(cGAN)の識別器は、畳み込みを用いたPatchGAN　classifierで、[pix2pix論文](https://arxiv.org/abs/1611.07004)にあるように、各画像の_patch_（パッチ：画像の一部分）から実画像かどうかを識別しようとするものです。


- 識別器の各ブロック内の構造: `畳み込み層 -> バッチ正規化 -> Leaky ReLU`
- 最後の出力層の後のテンソルの形は `(batch_size, 30, 30, 1)` です。
- 出力テンソルのの各 `30 x 30` の画像パッチは、入力画像の `70 x 70` の部分を分類するのに該当します。
  ※https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/issues/39
- 識別器には2つのペア画像が入力されます。
    - 入力画像とターゲット画像のペアを本物として分類されるべきものです。
    - 入力画像と生成画像（生成器（ジェネレーター）の出力）は偽物として分類されるべきものです。
    - これらのペア画像を作るのに連結する方法として、`tf.concat([inp, tar], axis=-1)`を用います。

識別器を定義する

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x)  # (batch_size, 128, 128, 64)
  down2 = downsample(128, 4)(down1)  # (batch_size, 64, 64, 128)
  down3 = downsample(256, 4)(down2)  # (batch_size, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

識別器（ディスクリミネーター)の可視化

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

生成器（ジェネレーター）をテストする

In [ ]:
disc_out = discriminator([inp[tf.newaxis, ...], gen_output], training=False)
plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

### 識別器（ディスクリミネーター)の損失関数


- `discriminator_loss` 関数では、**実画像** と **生成画像** の2つの入力を受け取ります。

- `real_loss` は、**実画像** と **1の配列（実画像＝本物のため）** のsigmoid cross-entropyロス（損失値）です。
- `generated_loss` は，**生成された画像** のsigmoid cross-entropyロス（損失値）と、**ゼロの配列（偽の画像）** です。
- `total_loss` は `real_loss` と `generated_loss` の合計です。

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

以下に図として識別モデルの学習手順を示します。

![Discriminator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/dis.png?raw=1)


※ネットワークアーキテクチャやハイパーパラメータの詳細については，[pix2pix](https://arxiv.org/abs/1611.07004)を参照してください。

## オプティマイザやチェックポイントを保存する関数の定義


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## 画像生成

学習中に画像を描画する関数を書きます。

- 複数のテスト画像データセットを生成器（ジェネレーター）に渡します。
- 生成器（ジェネレーター）は、入力画像を変換し出力します。
- 最後に予測画像を描画して、終わりです！

**注：**以下の`training=True`は意図的なものです。というのも、テストデータセット上でNNモデルを実行する際に、ある程度の統計情報のバッチ処理が必要だからです。もし `training=False` を使うと、学習データセットから学習された累積統計量を得ることになり、これは望ましくありません。

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

以上の関数を実行すると...

In [ ]:
for example_input, example_target in test_dataset.take(1):
  generate_images(generator, example_input, example_target)

## 学習

1.   各入力に対して、生成器により出力画像が生成されます。
2.   識別器は最初の入力として，`input_image`と生成された画像を受け取り、2番目の入力は`input_image`と`target_image`を受け取ります。
3. そして、生成器と識別器の損失を計算します。
4. その後、生成器と識別器の両方の入力に対するロス（損失）から勾配を計算し、それをオプティマイザ（最適化手法）に適用します。
5. 最後に、ロス（損失）をTensorBoardに随時記録していきます。

In [ ]:
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, step):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=step//1000)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=step//1000)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=step//1000)
    tf.summary.scalar('disc_loss', disc_loss, step=step//1000)

以下、実際の学習時のループです。このチュートリアルでは、複数のデータセットを扱うことができ、データセットのサイズも様々なので、学習時のループはepochではなく、step(イテレーション)で動作するように設定されています。

- stepの数だけ反復します。
- 10stepごとにドット(`.`)を表示します。
- 1000stepごとにディスプレイをクリアし、`generate_images`を実行して進捗状況を表示します。
- 5000stepごとにチェックポイントを保存します。

In [ ]:
def fit(train_ds, test_ds, steps):
  example_input, example_target = next(iter(test_ds.take(1)))
  start = time.time()

  for step, (input_image, target) in train_ds.repeat().take(steps).enumerate():
    if (step) % 1000 == 0:
      display.clear_output(wait=True)

      if step != 0:
        print(f'Time taken for 1000 steps: {time.time()-start:.2f} sec\n')

      start = time.time()

      generate_images(generator, example_input, example_target)
      print(f"Step: {step//1000}k")

    train_step(input_image, target, step)

    # Training step
    if (step+1) % 10 == 0:
      print('.', end='', flush=True)


    # Save (checkpoint) the model every 5k steps
    if (step + 1) % 5000 == 0:
      checkpoint.save(file_prefix=checkpoint_prefix)

この学習時のループではログが保存され、TensorBoardで表示して学習の進捗状況を確認することができます。

ローカルマシンで作業する場合は、別のTensorBoardプロセスを起動する必要があります。ノートブックで作業する場合は、TensorBoardで監視するために、学習を開始する前にビューアを起動する必要があります。

ビューアを起動するには、以下をコードセルに貼り付けます。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

最後に学習を走らせてみましょう。

In [ ]:
fit(train_dataset, test_dataset, steps=40000)

TensorBoardの結果を一般公開したい場合は、以下をコードセルにコピーして[TensorBoard.dev](https://tensorboard.dev/)にログをアップロードします。

**注：**これにはGoogleアカウントが必要です。

```
!tensorboard dev upload --logdir {log_dir}
```

**注：**また、上記のコマンドは終了しません。長時間にわたる実験の結果を継続的にアップロードするために設計されています。データがアップロードされたら、ノートブック内のランタイムタブからの「実行を中断」を選択して、停止する必要があります。


このノートブックの[以前の実行結果](https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw)を[TensorBoard.dev](https://tensorboard.dev/)で確認することができます。

TensorBoard.devは、MLの実験をホストし、トラッキングし、みんなで共有するために管理されたエクスペリエンスです。

`<iframe>`を使ってインラインで表示することもできます。

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
    width="100%",
    height="1000px")

GAN（またはpix2pixのようなcGAN）を学習する場合、単純な分類モデルや回帰モデルに比べて、損失などのログの解釈はより複雑になります。調べるべきこととして以下にまとめましたのでご参照ください。
<br>
- 生成器と識別器のどちらも「勝って」いないことを確認します。`gen_gan_loss` や `disc_loss` が非常に低い場合は、どちらかのモデルがもう一方のモデルを支配しており、モデル全体としての学習がうまくいっていないことを示しています。
- `log(2) = 0.69`という値は、これらの損失の良い参考になります。これはperplexity(情報量)が2であることを示しており、識別者は平均して2つの選択肢について同じように不確かであることを意味しています。
- `disc_loss`では、`0.69`以下の値は、実画像と生成画像を組み合わせにおいて、識別器がランダムに識別するよりも良い結果を出していることを意味します。
- `gen_gan_loss` では、`0.69`以下の値は生成器がランダムなものよりも識別器を欺くのに優れていることを意味します。
- 学習が進むにつれて、`gen_l1_loss` は減少するはずです。


## 最新のモデルのチェックポイントを復元し、ネットワークをテストする

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# Restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## テストデータセットを使って画像を生成する

In [ ]:
# Run the trained model on a few examples from the test set
for inp, tar in test_dataset.take(5):
  generate_images(generator, inp, tar)